In [2]:
# 출처
# https://dooyeoung.tistory.com/36

# 관련문서
# https://developers.google.com/analytics/devguides/reporting/core/v3

# 주요결과 sample 
# https://developers.google.com/analytics/devguides/reporting/core/v3/common-queries

+ [x] GA 접속
+ [x] test
+ [x] 주요지표 확인
  + [x] APP activation : 기술 > 브라우저 및 운영체제
  + [x] ALL activation : 획득 > 전체 트래픽 > 채널
  + [ ] DAU(일 평균)   : 잠재고객 > 개요 
  + [x] MAU(일 평균)   : 잠재고객 > 개요 
+ [x] GA 화면별 내용 출력
  + [ ] Monthly
    + 날짜생성(금년도)
    + for문 설정
    + pivot (or crosstab) 
  + [ ] Weekly
  

### GA 접속

In [4]:
# !pip install --upgrade google-api-python-client

In [1]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np
import pandas as pd

from datetime import datetime


C:\Users\MAEIL\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
def get_service(api_name, api_version, scopes, key_file_location):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scopes=scopes)
    service = build(api_name, api_version, credentials=credentials)
    return service
    
# Define the auth scopes to request.
scope = 'https://www.googleapis.com/auth/analytics.readonly'
key_file_location = './vision API-06a448b64428.json'

# Authenticate and construct service.
service = get_service(api_name='analytics', api_version='v3', scopes=[scope], key_file_location=key_file_location)

# Get a list of all Google Analytics accounts for this user
accounts = service.management().accounts().list().execute()
if accounts.get('items'):
    # Get the first Google Analytics account.
    account = accounts.get('items')[0].get('id')
    # Get a list of all the properties for the first account.
    properties = service.management().webproperties().list(accountId=account).execute()

### test
+ https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/

In [4]:
# test : 일주일간 세션수와, 페이지뷰수 받아오기
result = service.data().ga().get(
            ids='ga:236972743',
            start_date='7daysAgo',
            end_date='2022-08-02',
            metrics='ga:newvisits,ga:sessions,ga:pageviews').execute()

In [5]:
result

{'kind': 'analytics#gaData',
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:236972743&metrics=ga:newvisits,ga:sessions,ga:pageviews&start-date=7daysAgo&end-date=2022-08-02',
 'query': {'start-date': '7daysAgo',
  'end-date': '2022-08-02',
  'ids': 'ga:236972743',
  'metrics': ['ga:newvisits', 'ga:sessions', 'ga:pageviews'],
  'start-index': 1,
  'max-results': 1000},
 'itemsPerPage': 1000,
 'totalResults': 1,
 'selfLink': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:236972743&metrics=ga:newvisits,ga:sessions,ga:pageviews&start-date=7daysAgo&end-date=2022-08-02',
 'profileInfo': {'profileId': '236972743',
  'accountId': '187709388',
  'webPropertyId': 'UA-187709388-1',
  'internalWebPropertyId': '261287584',
  'profileName': '01. All Web Site Data',
  'tableId': 'ga:236972743'},
 'containsSampledData': False,
 'columnHeaders': [{'name': 'ga:newvisits',
   'columnType': 'METRIC',
   'dataType': 'INTEGER'},
  {'name': 'ga:sessions', 'columnType': 'METRIC', 'data

In [6]:
result['rows'][0][0], result['rows'][0][1], result['rows'][0][2]

('26224', '51885', '282921')

### 주요지표 확인
#### 기술 > 브라우저 및 운영체제 : APP activation(by GA)

In [7]:
# 조건1 : 일자
start_date_info = '2022-06-01'
end_date_info   = '2022-06-30'

# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:browser,ga:channelGrouping,ga:medium,ga:source'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 결과 출력
result = service.data().ga().get(ids='ga:236972743',
                                 start_date=start_date_info, end_date=end_date_info, 
                                 dimensions=dimensions_value, metrics=metrics_value).execute()

In [8]:
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")

df1 = pd.DataFrame(result['rows'], columns =column_name)
df1 = df1[(df1.browser == 'Android Webview' )]
# df1 = df1[(df1.browser == 'Android Webview' )|(df1.browser == 'Safari (in-app)')]
df1['end_date'] = end_date_info

In [9]:
df1[:40]

,browser,channelGrouping,medium,source,users,end_date
4,Android Webview,(Other),brandsum,naver,8,2022-06-30
5,Android Webview,(Other),conv_mo,instagram,18,2022-06-30
6,Android Webview,(Other),cpc,google,1385,2022-06-30
7,Android Webview,(Other),gfa_mo,naver,3,2022-06-30
8,Android Webview,(Other),gfa_pc,naver,1,2022-06-30
9,Android Webview,(Other),keyword,kakao,160,2022-06-30
10,Android Webview,(Other),mo_ct,mobon_da,6,2022-06-30
11,Android Webview,(Other),qr,product,18,2022-06-30
12,Android Webview,(Other),recommend,vuon,1,2022-06-30
13,Android Webview,Direct,(none),(direct),19242,2022-06-30


#### 획득 > 전체 트래픽 > 채널 : ALL activation(by GA)

In [61]:
# 기준 테이블 작성
df_order = pd.DataFrame()
df_order['channelGrouping'] = ['Display', 'Direct', 'Organic Search', 'Social', 'Paid Search', 'Referral', '(Other)']

In [66]:
# 조건1 : 일자
start_date_info = '2021-03-01'
end_date_info   = '2021-03-31'

# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:channelGrouping'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:sessions'

# 결과 출력
result = service.data().ga().get(ids='ga:236972743',
                                 start_date=start_date_info, end_date=end_date_info, 
                                 dimensions=dimensions_value, metrics=metrics_value).execute()

In [67]:
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df1 = pd.DataFrame(result['rows'], columns =column_name)
df1['end_date'] = end_date_info

In [68]:
df1 = pd.merge(df_order, df1, on ='channelGrouping', how ='left')

In [69]:
df1

,channelGrouping,sessions,end_date
0,Display,9251,2021-03-31
1,Direct,25348,2021-03-31
2,Organic Search,5343,2021-03-31
3,Social,2899,2021-03-31
4,Paid Search,17,2021-03-31
5,Referral,478,2021-03-31
6,(Other),1859,2021-03-31


#### 잠재고객 > 개요 : DAU(일 평균, 인원)

In [4]:
# 조건1 : 일자
start_date_info = '2021-04-01'
end_date_info   = '2021-04-30'

# 조건2 : 필터링 항목(dimensions)
metrics_value = ''

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users,ga:newUsers'

# 결과 출력
result = service.data().ga().get(ids='ga:236972743',
                                 start_date=start_date_info, end_date=end_date_info,
                                 metrics=metrics_value).execute()

In [5]:
column_name = (metrics_value).replace('ga:', '').split(",")
df1 = pd.DataFrame(result['rows'], columns =column_name)
df1['end_date'] = end_date_info

In [6]:
df1

,users,newUsers,end_date
0,72138,68365,2021-04-30


#### 잠재고객 > 개요 : MAU(일 평균, 인원)

In [37]:
# 조건1 : 일자
start_date_info = '2021-03-01'
end_date_info   = '2021-03-31'

# 조건2 : 필터링 항목(dimensions)
metrics_value = ''

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users,ga:newUsers'

# 결과 출력
result = service.data().ga().get(ids='ga:236972743',
                                 start_date=start_date_info, end_date=end_date_info, metrics=metrics_value).execute()

In [38]:
column_name = (metrics_value).replace('ga:', '').split(",")
df1 = pd.DataFrame(result['rows'], columns =column_name)
# df1['end_date'] = end_date_info

In [39]:
df1 = df1.T
df1['end_date'] = start_date_info

In [40]:
df1

,0,end_date
users,24133,2021-03-31
newUsers,20231,2021-03-31


'2021-04-22'

### GA 화면별 내용 출력
#### Monthly, APP activation

In [11]:
# 조건1 : 일자
start = "2021-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='MS')]
end_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='M')]
if len(start_date_info) != len(end_date_info):
    end_date_info.append(end)


In [329]:
# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:browser'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df1 = pd.DataFrame( columns =column_name)
df1['date'] = ''

# 결과 수집+정리
for i in range(0, len(start_date_info)):
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df1 = pd.concat([df1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df1['gubun'] = 'APP activation'

# 임시자료 추가(2021년 1월 미수집에 따른)
df_temp = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-01-01'], "gubun":['APP activation']})
df1 = df1.append(df_temp, ignore_index = True)

# 필요한 자료만 필터링
df1 = df1[(df1.browser == 'Android Webview' )|(df1.browser == 'Safari (in-app)')]

# header name 변경
df1.rename(columns = {'browser' : 'gubun2', 'users' : 'value'}, inplace = True)

In [200]:
df1.tail()

,gubun2,value,date,gubun
1,Android Webview,11427,2021-02-01,APP activation
9,Safari (in-app),4262,2021-02-01,APP activation
14,Android Webview,10217,2021-03-01,APP activation
23,Safari (in-app),2800,2021-03-01,APP activation
26,Android Webview,0,2021-01-01,APP activation


#### Monthly, ALL activation

In [330]:
# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:channelGrouping'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:sessions'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df2 = pd.DataFrame( columns =column_name)
df2['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df2 = pd.concat([df2, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df2['gubun'] = 'ALL activation'

# header name 변경
df2.rename(columns = {'channelGrouping' : 'gubun2', 'sessions' : 'value'}, inplace = True)

In [202]:
df2.tail()

,gubun2,value,date,gubun
2,Display,9251,2021-03-01,ALL activation
3,Organic Search,5343,2021-03-01,ALL activation
4,Paid Search,17,2021-03-01,ALL activation
5,Referral,478,2021-03-01,ALL activation
6,Social,2899,2021-03-01,ALL activation


#### Monthly, DAU

In [22]:
# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_1 = pd.DataFrame( columns =column_name)
df3_1['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff
        df_temp['date'] = start_date_info[i]
        df3_1 = pd.concat([df3_1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df3_1['gubun'] = 'DAU'
df3_1['gubun2'] = 'users'

# header name 변경
df3_1.rename(columns = {'users' : 'value'}, inplace = True)


# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:newUsers'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_2 = pd.DataFrame( columns =column_name)
df3_2['date'] = ''


for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff+1
        df_temp['date'] = start_date_info[i]
        df3_2 = pd.concat([df3_2, df_temp])
    except:
        pass
    
# 자료의 상위속성 추가
df3_2['gubun'] = 'DAU'
df3_2['gubun2'] = 'newUsers'
# header name 변경
df3_2.rename(columns = {'newUsers' : 'value'}, inplace = True)

df3 = pd.concat([df3_1, df3_2])

In [23]:
df3

,value,date,gubun,gubun2
0,730.703704,2021-02-01,DAU,users
0,804.433333,2021-03-01,DAU,users
0,2891.920000,2021-04-01,DAU,users
0,737.074074,2021-02-01,DAU,newUsers
0,675.366667,2021-03-01,DAU,newUsers
0,2739.880000,2021-04-01,DAU,newUsers


#### Monthly, MAU

In [450]:
# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df4_1 = pd.DataFrame( columns =column_name)
df4_1['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df4_1 = pd.concat([df4_1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df4_1['gubun'] = 'MAU'
df4_1['gubun2'] = 'users'

# header name 변경
df4_1.rename(columns = {'users' : 'value'}, inplace = True)


# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:newUsers'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df4_2 = pd.DataFrame( columns =column_name)
df4_2['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df4_2 = pd.concat([df4_2, df_temp])
    except:
        pass
    
# 자료의 상위속성 추가
df4_2['gubun'] = 'MAU'
df4_2['gubun2'] = 'newUsers'
# header name 변경
df4_2.rename(columns = {'newUsers' : 'value'}, inplace = True)

df4 = pd.concat([df4_1, df4_2])

In [451]:
df_temp

,newUsers,date
0,25875,2021-04-18


In [452]:
df4.tail()

,value,date,gubun,gubun2


### 최종 자료 : MTHLY

In [453]:
df_MTLY = pd.concat([df1,df2,df3,df4])
df_MTLY = pd.pivot_table(df_MTLY, index = ['gubun', 'gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0)

### GA 내용 출력 : Weekly

In [3]:
# 조건1 : 일자(weekly)
start = "2021-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='W')]
end_date_info   = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='W')+ pd.offsets.Day(6)]
# if len(start_date_info) != len(end_date_info):
#     start_date_info = start_date_info[:-1]

In [4]:
# end_date_info

In [5]:
#############
# Weekly, APP activation
#####

# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:browser'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df1 = pd.DataFrame( columns =column_name)
df1['date'] = ''

# 결과 수집+정리
for i in range(0, len(start_date_info)):
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df1 = pd.concat([df1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df1['gubun'] = 'APP activation'

# 임시자료 추가(2021년 1월 미수집에 따른)
df_temp = pd.DataFrame({"browser":['Android Webview', 'Android Webview', 'Android Webview','Android Webview','Android Webview'], 
                        "users":[0,0,0,0,0], 
                        "date":['2021-01-03', '2021-01-10', '2021-01-17', '2021-01-24', '2021-01-31',],
                        "gubun":['APP activation', 'APP activation','APP activation' ,'APP activation','APP activation']})
df1 = df1.append(df_temp, ignore_index = True)

# 필요한 자료만 필터링
df1 = df1[(df1.browser == 'Android Webview' )|(df1.browser == 'Safari (in-app)')]

# header name 변경
df1.rename(columns = {'browser' : 'gubun2', 'users' : 'value'}, inplace = True)



#############
# Weekly, ALL activation
#####
# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:channelGrouping'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:sessions'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df2 = pd.DataFrame( columns =column_name)
df2['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df2 = pd.concat([df2, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df2['gubun'] = 'ALL activation'

# header name 변경
df2.rename(columns = {'channelGrouping' : 'gubun2', 'sessions' : 'value'}, inplace = True)




#############
# Weekly, DAU
#####

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_1 = pd.DataFrame( columns =column_name)
df3_1['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff
        df_temp['date'] = start_date_info[i]
        df3_1 = pd.concat([df3_1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df3_1['gubun'] = 'DAU'
df3_1['gubun2'] = 'users'

# header name 변경
df3_1.rename(columns = {'users' : 'value'}, inplace = True)


# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:newUsers'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_2 = pd.DataFrame( columns =column_name)
df3_2['date'] = ''


for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff
        df_temp['date'] = start_date_info[i]
        df3_2 = pd.concat([df3_2, df_temp])
    except:
        pass
    
# 자료의 상위속성 추가
df3_2['gubun'] = 'DAU'
df3_2['gubun2'] = 'newUsers'
# header name 변경
df3_2.rename(columns = {'newUsers' : 'value'}, inplace = True)

df3 = pd.concat([df3_1, df3_2])



#############
# Weekly, MAU
#####
# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df4_1 = pd.DataFrame( columns =column_name)
df4_1['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df4_1 = pd.concat([df4_1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df4_1['gubun'] = 'WAU'
df4_1['gubun2'] = 'users'

# header name 변경
df4_1.rename(columns = {'users' : 'value'}, inplace = True)


# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:newUsers'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df4_2 = pd.DataFrame( columns =column_name)
df4_2['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_info[i]
        df4_2 = pd.concat([df4_2, df_temp])
    except:
        pass
    
# 자료의 상위속성 추가
df4_2['gubun'] = 'WAU'
df4_2['gubun2'] = 'newUsers'
# header name 변경
df4_2.rename(columns = {'newUsers' : 'value'}, inplace = True)

df4 = pd.concat([df4_1, df4_2])

In [6]:
df_WKYL = pd.concat([df1,df2,df3,df4])
df_WKYL = pd.pivot_table(df_WKYL, index = ['gubun', 'gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0)

In [7]:
df_WKYL

date                            2021-01-03  2021-01-10  2021-01-17  \
gubun          gubun2                                                
ALL activation (Other)                   0           0           0   
               Direct                    0           0           0   
               Display                   0           0           0   
               Organic Search            0           0           0   
               Paid Search               0           0           0   
               Referral                  0           0           0   
               Social                    0           0           0   
APP activation Android Webview           0           0           0   
               Safari (in-app)           0           0           0   
DAU            newUsers                  0           0           0   
               users                     0           0           0   
WAU            newUsers                  0           0           0   
               users                     0           0           0   

date                            2021-01-24 2021-01-31 2021-02-07 2021-02-14  \
gubun          gubun2                                                         
ALL activation (Other)                   0          0          0          0   
               Direct                    0          0      14036      23309   
               Display                   0          0          0          0   
               Organic Search            0          0        414       1148   
               Paid Search               0          0          0          0   
               Referral                  0          0         51        101   
               Social                    0          0        394        773   
APP activation Android Webview           0          0       4938       6768   
               Safari (in-app)           0          0       1918       2554   
DAU            newUsers                  0          0    1306.33    1303.83   
               users                     0          0       1292    1837.67   
WAU            newUsers                  0          0       7838       7823   
               users                     0          0       7752      11026   

date                           2021-02-21 2021-02-28 2021-03-07 2021-03-14  \
gubun          gubun2                                                        
ALL activation (Other)                  1        102        173        961   
               Direct                8548       7687       5479       5558   
               Display                  0          3       8416        533   
               Organic Search         966       1170       1011       1082   
               Paid Search            311         20          2          0   
               Referral               105        112        116        107   
               Social                 904        723        623        578   
APP activation Android Webview       3625       3178       4540       2619   
               Safari (in-app)       1202        939        863        803   
DAU            newUsers           626.167        576    1618.17      502.5   
               users              1072.83    974.167    1935.83        864   
WAU            newUsers              3757       3456       9709       3015   
               users                 6437       5845      11615       5184   

date                           2021-03-21 2021-03-28 2021-04-04 2021-04-11  \
gubun          gubun2                                                        
ALL activation (Other)                469        447        222      18612   
               Direct                4831       4628       3305       6732   
               Display                227        134         94       2619   
               Organic Search        1340       1579       1352       2122   
               Paid Search              0          0          0          0   
               Referral               103     

### 일별 자료

In [470]:
# 조건1 : 일자(weekly)
start = "2021-04-01"
end = datetime.today().strftime('%Y-%m-%d')
start_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='D')]
end_date_info   = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='D')+ pd.offsets.Day(1)]

In [ ]:
# end_date_info

In [472]:
#############
# Weekly, DAU
#####

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_1 = pd.DataFrame( columns =column_name)
df3_1['date'] = ''

for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff
        df_temp['date'] = start_date_info[i]
        df3_1 = pd.concat([df3_1, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df3_1['gubun'] = 'DAU'
df3_1['gubun2'] = 'users'

# header name 변경
df3_1.rename(columns = {'users' : 'value'}, inplace = True)


# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:newUsers'

# 수집내용 정의
column_name = ( metrics_value).replace('ga:', '').split(",")
df3_2 = pd.DataFrame( columns =column_name)
df3_2['date'] = ''


for i in range(0, len(start_date_info)):
    # 결과 출력
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         metrics=metrics_value).execute()
        day_diff = (datetime.strptime(end_date_info[i], "%Y-%m-%d") - datetime.strptime(start_date_info[i], "%Y-%m-%d")).days
        df_temp = pd.DataFrame(result['rows'], columns =column_name).astype(int)/day_diff
        df_temp['date'] = start_date_info[i]
        df3_2 = pd.concat([df3_2, df_temp])
    except:
        pass
    
# 자료의 상위속성 추가
df3_2['gubun'] = 'DAU'
df3_2['gubun2'] = 'newUsers'
# header name 변경
df3_2.rename(columns = {'newUsers' : 'value'}, inplace = True)

df3 = pd.concat([df3_1, df3_2])


In [ ]:
pd.pivot_table(df3, index = ['gubun', 'gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0).T